# Práctica 6
## Propuesta: Intento de sistema de reconocimiento facial y reacciones según datos de DeepFace
<ul>
<li>Nikhil Chandru Durgadas Chellaram</li>
<li>Raúl Mateus Sánchez</li>
</ul>

Para comprobar el desarrollo de la práctica: https://github.com/raulmat19/Vision-por-Computador/tree/main/Practica-6

In [9]:
import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt
import os
import numpy as np

In [10]:
models = ["VGG-Face","Facenet","Facenet512","OpenFace","DeepFace","DeepID","ArcFace","Dlib","SFace"]

metrics = ["cosine", "euclidean", "euclidean_l2"]

backends = ['opencv','ssd','dlib','mtcnn','retinaface','mediapipe']

In [11]:
def draw_text(img, text,
          font=cv2.FONT_HERSHEY_SIMPLEX,
          pos=(0, 0),
          font_scale=3,
          font_thickness=2,
          text_color=(255, 255, 255),
          text_color_bg=(0, 0, 0)
          ):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w + 5, y + text_h + 5), text_color_bg, -1)
    cv2.putText(img, text, (x, int(y + text_h + font_scale - 1)), font, font_scale, text_color, font_thickness)

    return text_size

Con este pequeño sistema, se permite al usuario registrar su cara en el sistema con la finalidad de simular un sistema de reconocimiento facial así como prodecer a las reacciones propuestas empleando las utilidades de DeepFace

In [12]:
def getFace(path, name):
    vid = cv2.VideoCapture(0)
    counter = 0

    while(True):      
        # fotograma a fotograma
        ret, frame = vid.read()

        if ret:
            draw_text(frame, f"{counter}", font_scale=1, pos=(50, 10), text_color_bg=(0, 0, 0))
            cv2.imshow('Cam', frame)   

        key = cv2.waitKey(5)     
        # Detenemos pulsado ESC
        if key and key == 27 or counter == 3:
            break
        elif key and key == 32:
            if counter > 0:
                new_path = path + '/' + name + str(counter) + '.jpg'
            
            else:
                new_path = path + '/' + name + '.jpg'

            cv2.imwrite(new_path, frame)
            counter += 1
            
            
    # Libera el objeto de captura
    vid.release()
    # Destruye ventanas
    cv2.destroyAllWindows()

In [13]:
face_loaded = False

data_loaded = input ("Si su cara ha sido ya registrada, introduzca 'Si'. En caso contrario, introduzca 'No' o cualquier cosa")
if data_loaded == 'Si' or data_loaded == 'si':
    face_loaded = True

if not face_loaded:
    data = False
    escape = 0

    while not data:

        name = input ("Introduce tu nombre: ")
        surname = input ("Introduce tus apellidos: ")
        age = input ("Introduce tu edad: ")
        sexo = input ("Introduce tu sexo: ")
        workingStatus = input ("Introduce tu ocupación: ")

        if name == '' or surname == '' or age == '' or sexo == '' or workingStatus == '':
            warning = input ("Datos incorrectos. Pulse Enter para volver a introducirlos")
            escape += 1
        
        else:
            data = True

        if escape >= 2:
            break

    if data:
        try:
            for file in os.listdir("face_recognition_images/"):
                if file.startswith("representations"):
                    os.remove("face_recognition_images/" + file)
                    
            dir_path = 'face_recognition_images/' + str(name)
            try:
                os.stat(dir_path)
            except:
                os.mkdir(dir_path)
            file_path = dir_path + '/info.txt'
            with open(file_path, 'w') as f:
                f.write(f"{name}, {surname}, {age}, {sexo}, {workingStatus}")
            f.close()

            warning = input ("Pulse Enter para escanear su cara. Debe sacarse tres fotos, empleando para ello la tecla 'Barra espaciadora'")
            getFace(dir_path, name)

        except:
            print("Fallo del sistema")

Sistema de reconocimiento facial, que compara la cara en tiempo real con las caras ya escaneadas y guardadas, y muestra la información del usuario

https://stackoverflow.com/questions/60674501/how-to-make-black-background-in-cv2-puttext-with-python-opencv

In [14]:
def showFlag(ruta, img):
    img_flag = cv2.imread(ruta)
    img_flag = cv2.resize(img_flag, (150, 100))
    draw_text(img, "En el Mundial apoyas a: ", font_scale=1, pos=(50, img.shape[0]-50), text_color_bg=(0, 0, 0))
    img[img.shape[0]-100:img.shape[0], img.shape[1]-150:img.shape[1]] = img_flag

In [15]:
def getFlag(raza):
    if raza == "white":
        return "flags_images/white"
    elif raza == "asian":
        return "flags_images/asian"
    elif raza == "indian":
        return "flags_images/indian"
    elif raza == "middle eastern":
        return "flags_images/middle-eastern"
    elif raza == "latino hispanic":
        return "flags_images/latino-hispanic"
    elif raza == "black":
        return "flags_images/black"

In [17]:
#%%capture
vid = cv2.VideoCapture(0)
counter = 0
init = False
flag_mode = False
line = None
y0, dy = 10, 4
properties = ["Nombre", "Apellidos", "Edad", "Sexo", "Ocupacion"]
prev_race_path = None
prev_race = None

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:
        # Sistema de reconocimiento facial y muestra de información de usuario
        if not flag_mode:
            if counter == 50 or not init:
                cv2.imwrite('cap_faces/face.jpg', frame)

                try:
                    df = DeepFace.find(img_path = 'cap_faces/face.jpg', db_path = "face_recognition_images/", model_name=models[2], distance_metric = metrics[2], prog_bar = False, enforce_detection=False, silent = True)
                    path = df.at[0, 'identity']
                    info_path = os.path.dirname(path) + "/info.txt"
                    with open(info_path) as f:
                        line = f.readline()
                    f.close()

                    draw_text(frame, "Desbloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))
                    for i, line in enumerate(line.split(", ")):
                        y = y0 + i*dy*7
                        draw_text(frame, f"{properties[i]}:  {line}", font_scale=1, pos=(5, y), text_color_bg=(0, 0, 0))
                    
                    counter == 0
                    init = True

                except:
                    init = False
                    print("Not recognised")
                    draw_text(frame, "Bloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))

            elif init and counter != 50:
                
                draw_text(frame, "Desbloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))
                for i, line in enumerate(line.split(", ")):
                    y = y0 + i*dy*7
                    draw_text(frame, f"{properties[i]}:  {line}", font_scale=1, pos=(5, y), text_color_bg=(0, 0, 0))
                
                counter += 1

        elif flag_mode and init:
            print("a")
            cv2.imwrite('cap_faces/frame.jpg', frame)
            obj = DeepFace.analyze(img_path = 'cap_faces/frame.jpg', actions = ['age', 'gender', 'race', 'emotion'], enforce_detection=False)
            race = obj['dominant_race']
            if race != prev_race:
                path_to_flag = getFlag(race)
                flag_list = os.listdir(path_to_flag)
                index = np.random.randint(0, len(flag_list))
                path_to_flag = path_to_flag + "/" + flag_list[index]
                showFlag(path_to_flag, frame)
                prev_race_path = path_to_flag
                prev_race = race

            else:
                showFlag(prev_race_path, frame)

            draw_text(frame, f"Raza: {race}", font_scale=1, pos=(0, frame.shape[0]-470), text_color_bg=(0, 0, 0))

        elif flag_mode and not init:
            draw_text(frame, "Accion bloqueada hasta identificacion.", font_scale=1, pos=(0, frame.shape[0]-80), text_color_bg=(0, 0, 0))
            draw_text(frame, "Pulse d para ello", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))
        cv2.imshow('Cam', frame)
                    
    # Detenemos pulsado ESC
    key = cv2.waitKey(5)
    if key == 27:
        break
    if key == ord('d'):
        if flag_mode:
            flag_mode = False
            init = False
        else:
            flag_mode = True
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 81ms/step


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 66ms/step


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Action: emotion:  75%|███████▌  | 3/4 [00:01<00:00,  2.11it/s]

1/1 [==============================] - 0s 103ms/step


Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  2.40it/s]


NameError: name 'ruta' is not defined

: 

En desarrollo ...